In [2]:
import numpy as np

In [ ]:
import numpy as np
import pandas as pd
import time
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')


# Our optimized implementation
class XGBoostNode:
    """Optimized XGBoost Node with histogram-based split finding"""
    
    def __init__(self, x, gradient, hessian, idxs, 
                 max_depth=6, min_child_weight=1, reg_lambda=1, 
                 gamma=0, min_leaf=1, learning_rate=0.3, depth=0,
                 max_bins=32):
        # Data
        self.x = x
        self.gradient = gradient
        self.hessian = hessian
        self.idxs = idxs
        self.depth = depth
        
        # Config parameters
        self.max_depth = max_depth
        self.min_child_weight = min_child_weight
        self.reg_lambda = reg_lambda
        self.gamma = gamma
        self.min_leaf = min_leaf
        self.learning_rate = learning_rate
        self.max_bins = max_bins
        
        # Node properties
        self.n_samples = len(idxs)
        self.n_features = x.shape[1]
        self.row_count = len(idxs)
        
        # Split properties
        self.score = float('-inf')
        self.feature_idx = None
        self.split = None
        
        # Tree structure
        self.lhs = None
        self.rhs = None
        self.val = self.compute_gamma(gradient[idxs], hessian[idxs])
        
        # Cache for speed
        self._histograms = {}
        self._bin_edges = {}
    
    def compute_gamma(self, gradient, hessian):
        """Compute optimal weight for a leaf node"""
        hessian_sum = np.sum(hessian)
        if hessian_sum == 0:
            return 0.0
        return -np.sum(gradient) / (hessian_sum + self.reg_lambda)
    
    @property
    def is_leaf(self):
        """Check if node is leaf"""
        return (self.score <= 0 or 
                self.depth >= self.max_depth or 
                self.row_count < self.min_leaf or
                self.feature_idx is None)
    
    def create_histograms(self):
        """Pre-compute histograms for all features - VECTORIZED"""
        if self._histograms:  # Already computed
            return
            
        for feature_idx in range(self.n_features):
            x_feature = self.x[self.idxs, feature_idx]
            grad_feature = self.gradient[self.idxs]
            hess_feature = self.hessian[self.idxs]
            
            # Create bins using quantiles (weighted by hessian)
            if len(np.unique(x_feature)) <= self.max_bins:
                bin_edges = np.unique(x_feature)
            else:
                # Weighted quantiles approximation
                sorted_idx = np.argsort(x_feature)
                sorted_x = x_feature[sorted_idx]
                sorted_hess = hess_feature[sorted_idx]
                
                # Cumulative hessian weights
                cum_hess = np.cumsum(sorted_hess)
                total_hess = cum_hess[-1]
                
                if total_hess > 0:
                    # Find quantile positions
                    quantile_positions = np.linspace(0, total_hess, self.max_bins + 1)
                    bin_edges = np.interp(quantile_positions, cum_hess, sorted_x)
                    bin_edges = np.unique(bin_edges)
                else:
                    bin_edges = np.unique(x_feature)
            
            self._bin_edges[feature_idx] = bin_edges
            
            # Build histogram for this feature
            if len(bin_edges) > 1:
                # Digitize values to bins
                bin_indices = np.digitize(x_feature, bin_edges) - 1
                bin_indices = np.clip(bin_indices, 0, len(bin_edges) - 2)
                
                # Aggregate gradients and hessians by bins
                hist_grad = np.zeros(len(bin_edges) - 1)
                hist_hess = np.zeros(len(bin_edges) - 1)
                hist_count = np.zeros(len(bin_edges) - 1)
                
                # Vectorized aggregation
                np.add.at(hist_grad, bin_indices, grad_feature)
                np.add.at(hist_hess, bin_indices, hess_feature)
                np.add.at(hist_count, bin_indices, 1)
                
                self._histograms[feature_idx] = {
                    'grad': hist_grad,
                    'hess': hist_hess,
                    'count': hist_count,
                    'edges': bin_edges
                }
    
    def find_varsplit_fast(self):
        """Fast split finding using pre-computed histograms"""
        self.create_histograms()
        
        for feature_idx in range(self.n_features):
            if feature_idx not in self._histograms:
                continue
                
            hist = self._histograms[feature_idx]
            self.histogram_search(feature_idx, hist)
    
    def histogram_search(self, feature_idx, hist):
        """Search best split using histogram - VECTORIZED"""
        grad_hist = hist['grad']
        hess_hist = hist['hess']
        count_hist = hist['count']
        bin_edges = hist['edges']
        
        n_bins = len(grad_hist)
        if n_bins <= 1:
            return
        
        # Vectorized computation for all possible splits
        # Left side: cumulative sum up to each split
        left_grad = np.cumsum(grad_hist)[:-1]  # Don't include last bin
        left_hess = np.cumsum(hess_hist)[:-1]
        left_count = np.cumsum(count_hist)[:-1]
        # index 0 → sau bin 1 → left = g1

        # index 1 → sau bin 2 → left = g1+g2

        # index 2 → sau bin 3 → left = g1+g2+g3
        
        # Right side: total - left
        total_grad = np.sum(grad_hist)
        total_hess = np.sum(hess_hist)
        total_count = np.sum(count_hist)
        
        right_grad = total_grad - left_grad
        right_hess = total_hess - left_hess  
        right_count = total_count - left_count
        
        # Filter valid splits
        valid_mask = ((left_count >= self.min_leaf) & 
                      (right_count >= self.min_leaf) &
                      (left_hess >= self.min_child_weight) &
                      (right_hess >= self.min_child_weight) &
                      (left_hess > 0) & (right_hess > 0))
        
        if not np.any(valid_mask):
            return
        
        # Vectorized gain computation
        left_score = left_grad**2 / (left_hess + self.reg_lambda)
        right_score = right_grad**2 / (right_hess + self.reg_lambda)
        parent_score = (total_grad**2) / (total_hess + self.reg_lambda)
        
        gains = 0.5 * (left_score + right_score - parent_score) - self.gamma
        
        # Apply valid mask
        gains = np.where(valid_mask, gains, float('-inf'))
        
        # Find best split
        best_bin_idx = np.argmax(gains)
        best_gain = gains[best_bin_idx]
        
        if best_gain > self.score and best_gain > 0:
            self.score = best_gain
            self.feature_idx = feature_idx
            # Split at bin edge
            self.split = bin_edges[best_bin_idx + 1]
    
    def build_tree(self):
        """Build tree with optimizations"""
        # Check stopping criteria
        if self.depth >= self.max_depth or self.row_count < self.min_leaf:
            return
            
        # Fast split finding
        self.find_varsplit_fast()
        
        # If no good split found, remain as leaf
        if self.score <= 0 or self.feature_idx is None:
            return
            
        # Create child nodes
        x_split = self.x[self.idxs, self.feature_idx]
        lhs_mask = x_split <= self.split
        rhs_mask = x_split > self.split
        
        lhs_idxs = self.idxs[lhs_mask]
        rhs_idxs = self.idxs[rhs_mask]
        
        # Create children
        if len(lhs_idxs) > 0:
            self.lhs = XGBoostNode(
                self.x, self.gradient, self.hessian, lhs_idxs,
                self.max_depth, self.min_child_weight, self.reg_lambda, 
                self.gamma, self.min_leaf, self.learning_rate, 
                self.depth + 1, self.max_bins
            )
            self.lhs.build_tree()
            
        if len(rhs_idxs) > 0:
            self.rhs = XGBoostNode(
                self.x, self.gradient, self.hessian, rhs_idxs,
                self.max_depth, self.min_child_weight, self.reg_lambda, 
                self.gamma, self.min_leaf, self.learning_rate, 
                self.depth + 1, self.max_bins
            )
            self.rhs.build_tree()
    
    def predict(self, X):
        """Predict for multiple samples"""
        return np.array([self.predict_row(xi) for xi in X])
    
    def predict_row(self, xi):
        """Predict single sample"""
        if self.is_leaf:
            return self.val * self.learning_rate

        if self.feature_idx is None:
            return self.val * self.learning_rate
            
        node = self.lhs if xi[self.feature_idx] <= self.split else self.rhs
        if node is None:
            return self.val * self.learning_rate
            
        return node.predict_row(xi)





In [16]:
class XGBoostRegressor:
    """Complete XGBoost Regressor implementation"""
    
    def __init__(self, n_estimators=100, max_depth=6, learning_rate=0.3,
                 min_child_weight=1, reg_lambda=1, gamma=0, min_leaf=1,
                 max_bins=32, early_stopping_rounds=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.min_child_weight = min_child_weight
        self.reg_lambda = reg_lambda
        self.gamma = gamma
        self.min_leaf = min_leaf
        self.max_bins = max_bins
        self.early_stopping_rounds = early_stopping_rounds
        
        self.trees = []
        self.base_prediction = 0.0
    
    def _compute_gradient_hessian(self, y_true, y_pred):
        """Compute gradients and hessians for squared loss"""
        gradient = 2 * (y_pred - y_true)
        hessian = 2 * np.ones_like(y_true)
        return gradient, hessian
    
    def fit(self, X, y, eval_set=None, verbose=True):
        """Fit the XGBoost model"""
        # Initialize base prediction (mean of targets)
        self.base_prediction = np.mean(y)
        y_pred = np.full(len(y), self.base_prediction, dtype=np.float64)
        
        # Validation data for early stopping
        if eval_set is not None:
            X_val, y_val = eval_set
            y_val_pred = np.full(len(y_val), self.base_prediction, dtype=np.float64)
            best_val_loss = float('inf')
            no_improvement_count = 0
        
        self.trees = []
        
        for round_num in range(self.n_estimators):
            # Compute gradients and hessians
            gradient, hessian = self._compute_gradient_hessian(y, y_pred)
            
            # Fit tree
            tree_params = {
                'max_depth': self.max_depth,
                'min_child_weight': self.min_child_weight,
                'reg_lambda': self.reg_lambda,
                'gamma': self.gamma,
                'min_leaf': self.min_leaf,
                'learning_rate': self.learning_rate,
                'max_bins': self.max_bins
            }
            
            n_samples = X.shape[0]
            idxs = np.arange(n_samples)
            
            tree = XGBoostNode(X, gradient, hessian, idxs, **tree_params)
            tree.build_tree()
            
            # Update predictions
            tree_pred = tree.predict(X)
            y_pred += tree_pred
            
            self.trees.append(tree)
            
            # Compute training loss
            train_loss = mean_squared_error(y, y_pred)
            
            # Validation and early stopping
            if eval_set is not None:
                val_tree_pred = tree.predict(X_val)
                y_val_pred += val_tree_pred
                val_loss = mean_squared_error(y_val, y_val_pred)
                
                if verbose and round_num % 10 == 0:
                    print(f"Round {round_num:3d}: Train MSE = {train_loss:.6f}, Val MSE = {val_loss:.6f}")
                
                # Early stopping logic
                if self.early_stopping_rounds:
                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        no_improvement_count = 0
                    else:
                        no_improvement_count += 1
                        if no_improvement_count >= self.early_stopping_rounds:
                            print(f"Early stopping at round {round_num}")
                            break
            else:
                if verbose and round_num % 10 == 0:
                    print(f"Round {round_num:3d}: Train MSE = {train_loss:.6f}")
    
    def predict(self, X):
        """Predict using all fitted trees"""
        predictions = np.full(X.shape[0], self.base_prediction, dtype=np.float64)
        
        for tree in self.trees:
            predictions += tree.predict(X)
        
        return predictions

In [9]:
!pip install xgboost


In [10]:
import xgboost as xgb

In [17]:

# Load data
data = fetch_california_housing()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model 1: XGBoostNode
print("\n1. XGBoostNode")
start_time = time.time()
model1 = XGBoostRegressor(n_estimators=50, max_depth=4, learning_rate=0.1, gamma=0.1)
model1.fit(X_train_scaled, y_train)
time1 = time.time() - start_time

pred1 = model1.predict(X_test_scaled)
mse1 = mean_squared_error(y_test, pred1)
r2_1 = r2_score(y_test, pred1)

print(f"  Time: {time1:.2f}s")
print(f"  MSE:  {mse1:.4f}")
print(f"  R²:   {r2_1:.4f}")

# Model 2: XGBoost Framework
print("\n2. XGBoost Framework")
start_time = time.time()
model2 = xgb.XGBRegressor(n_estimators=50, max_depth=4, learning_rate=0.1, gamma=0.1, random_state=42)
model2.fit(X_train_scaled, y_train)
time2 = time.time() - start_time

pred2 = model2.predict(X_test_scaled)
mse2 = mean_squared_error(y_test, pred2)
r2_2 = r2_score(y_test, pred2)

print(f"  Time: {time2:.2f}s")
print(f"  MSE:  {mse2:.4f}")
print(f"  R²:   {r2_2:.4f}")

# So sánh
print(f"\n3. So sánh:")
print(f"  MSE difference: {abs(mse1 - mse2):.4f}")
print(f"  R² difference:  {abs(r2_1 - r2_2):.4f}")
print(f"  Time ratio:     {time1/time2:.2f}x")



1. XGBoostNode
Round   0: Train MSE = 1.188244
Round  10: Train MSE = 0.555961
Round  20: Train MSE = 0.396379
Round  30: Train MSE = 0.323572
Round  40: Train MSE = 0.291180
  Time: 6.49s
  MSE:  0.3015
  R²:   0.7699

2. XGBoost Framework
  Time: 0.60s
  MSE:  0.3009
  R²:   0.7704

3. So sánh:
  MSE difference: 0.0006
  R² difference:  0.0004
  Time ratio:     10.73x
